# PS6: Drug Combination Design using Reinforcement Learning
In this problem set, we apply Q-learning to design drug combinations that maximize therapeutic effectiveness while satisfying budget constraints. The goal is to learn an optimal policy that selects drug types and dosage levels through repeated interaction with an environment.

> __Learning Objectives:__
> 
> By the end of this lab, you will be able to:
> * **Implement Q-learning for constrained optimization**: Build a Q-learning agent that learns state-action value functions through temporal difference updates, demonstrating how reinforcement learning discovers optimal policies without explicit knowledge of environment dynamics.
> * **Model combinatorial drug design problems**: Encode drug cocktail design as a Markov decision process with discrete state spaces (dosage levels) and action spaces (drug selection), and apply Cobb-Douglas utility functions to evaluate therapeutic effectiveness under budget constraints.
> * **Extract policies from learned Q-values**: Derive optimal drug selection and dosage policies by analyzing converged Q-value tables, mapping each state (current dosage configuration) to the action (drug combination) that maximizes expected cumulative reward.

In this problem set, you will implement the __experiment__ function that our Q-learning agent will interact with to learn an optimal drug combination policy.

Let's get started!
___

## Problem
The Drug Cocktail Design Problem involves selecting a combination of drugs to create a cocktail that maximizes therapeutic effectiveness while staying within budget and safety constraints. Each drug has attributes such as dosage, mechanism of action, and side effects, which contribute to the overall effectiveness of the cocktail. The drug cocktail design problem can be formulated as the following optimization problem:
$$
\boxed{
\begin{align*}
\text{maximize} \quad & U(n_1, \dots, n_K) = \kappa(\gamma) \prod_{i=1}^{K} n_i^{\gamma_i} \\
\text{subject to} \quad & \sum_{i=1}^{K} c_i (n_i\;W) \leq B\quad\text{(budget constraint)}\\
& \sum_{i=1}^{K} n_i \leq S\quad\text{(safety constraint)}\\
& n_{i}^{\text{min}} \leq n_i \leq n_i^{\text{max}} \quad \forall i \in \{1, \dots, K\}
\end{align*}}
$$
where the objective function is a Cobb-Douglas utility function representing the effectiveness of the drug cocktail, and the constraints ensure that the cocktail stays within budget and safety limits. The design variables are the concentrations of each drug in the cocktail, denoted by $n_i$ for drug $i$ (units: mg/kg), $K$ is the total number of available drugs, and $W$ is the patient weight (units: kg). The $\kappa(\gamma)$ term is defined as:
$$
\begin{align*}
\kappa(\gamma) & = \begin{cases}
-1 & \text{if any } \gamma_i < 0 \\
1 & \text{if all } \gamma_i \geq 0
\end{cases}
\end{align*}
$$
where coefficients $\gamma_i$ represent the efficacy of each drug type, which must be learned from historical data or set based on expert knowledge.

The first set of constraints ensures that the total cost of the drug cocktail does not exceed a specified budget $B$, where $c_i$ is the cost per unit concentration (e.g., USD/mg) of drug $i$. The second set of constraints ensures that the total concentration of all drugs in the cocktail does not exceed a safety limit $S$. Finally, each drug concentration is bounded by minimum and maximum allowable levels, denoted by $n_i^{\text{min}}$ and $n_i^{\text{max}}$, respectively.

In this lab, we'll implement the budget constraint only.
___

## Q-Learning Algorithm
Q-learning iteratively estimates the state action-value function $Q(s, a)$ by conducting repeated experiments $t=1,2,\ldots$ in the world $\mathcal{W}$. 
In each experiment, an agent in state $s\in\mathcal{S}$ takes action $a\in\mathcal{A}$, receives a reward $r$, and (potentially) transitions to a new state $s^{\prime}$. After each experiment $t$, the agent updates its estimate of $Q(s, a)$ using the update rule:
$$
\begin{equation*}
Q_{t+1}(s,a)\leftarrow{\underbrace{Q_{t}(s,a)}_{\text{old value}}}+\alpha_{t}\cdot\underbrace{\left(r+\gamma\cdot\max_{a^{\prime}\in\mathcal{A}}Q_{t}(s^{\prime},a^{\prime}) - Q_{t}(s,a)\right)}_{\text{new value}}\quad{t = 1,2,3,\ldots}
\end{equation*}
$$
where $0<\alpha_{t} <{1}$ is the learning rate parameter at time $t$, and $0<\gamma<{1}$ is the discount factor. 
We estimate the policy function $\pi:\mathcal{S}\rightarrow\mathcal{A}$ by selecting the action $a$ that maximizes $Q(s,a)$ at each state $s$:
$$
\begin{equation*}
\pi(s) = \arg\max_{a\in\mathcal{A}}Q(s,a)
\end{equation*}
$$

### Algorithm
Initialize $Q(s,a)$ arbitrarily for all $s\in\mathcal{S}$, and $a\in\mathcal{A}$.
Set the hyperparameters: learning rate $\alpha_{t}$, the discount factor $\gamma$, the exploration rate $\epsilon_{t}$, the maximum number of iterations $\texttt{maxiter}$, and the convergence tolerance $\delta$. Set $\texttt{converged}\gets\texttt{false}$. 

For $s\in\mathcal{S}$
1. Initialize the trial counter $t\gets{1}$
2. While $\texttt{converged} $ is $\texttt{false}$ __do__:
    1. Roll a random number $p\in[0,1]$. Compute $\epsilon_{t}={t^{-1/3}}\cdot\left(K\cdot\log(t)\right)^{1/3}$ where $K=|\mathcal{A}|$ is the number of actions.
    2. If $p\leq\epsilon_{t}$, choose a random (uniform) action $a_{t}\in\mathcal{A}$. Otherwise, choose a greedy action $a_{t} = \text{arg}\max_{a\in\mathcal{A}}{Q_{t}(s,a)}$.
    3. Take action $a_{t}$, observe the reward $r$ from the __world__ and transition to the next state $s^{\prime}$.
    4. Update the state-action-value function: $Q_{t+1}(s,a)\leftarrow{Q_{t}(s,a)}+\alpha_{t}\cdot\underbrace{\left(r+\gamma\cdot\overbrace{\max_{a^{\prime}\in\mathcal{A}}Q_{t}(s^{\prime},a^{\prime})}^{\text{one-step lookahead}} - Q_{t}(s,a)\right)}_{\text{new information}}$.
    5. Update the state $s\leftarrow{s^{\prime}}$, the learning rate $\alpha_{t+1}\leftarrow\alpha_{t}$, and the counter $t\leftarrow{t+1}$
    6. Convergence check: If the Q-table has bounded change $\lVert{Q_{t+1} - Q_{t}}\rVert\leq\delta$, then the algorithm has converged. Set $\texttt{converged}\gets\texttt{true}$.
    7. Otherwise: if $t\geq\texttt{maxiter}$, then set $\texttt{converged}\gets\texttt{true}$ and notify the caller that the maximum iteration limit was reached without convergence. Proceed to next state.
    8. Otherwise: continue to the next iteration.
3. End While
4. End For

### Convergence
Q-learning converges to the optimal policy under two key theoretical conditions (assuming the Markov property holds for the environment):
* __Learning rate decay__: The learning rate $\alpha_{t}$ must satisfy $\sum_{t=0}^\infty \alpha_t(s, a) = \infty$ and $\sum_{t=0}^\infty \alpha_t^2(s, a) < \infty$ for all state-action pairs, ensuring sufficient initial updates while stabilizing over time. Setting $\alpha_{t+1} \gets \beta\alpha_{t}$ where $\beta<1$ is a common choice.
* __Infinite exploration__: All state-action pairs must be visited infinitely often. This condition holds for $\epsilon$-greedy policies with persistent exploration, i.e., $\epsilon_{t} > 0\,\,\forall{t}$.
___

## Setup, Data, and Prerequisites
First, we set up the computational environment by including the `Include.jl` file and loading any needed resources.

> The [`include(...)` command](https://docs.julialang.org/en/v1/base/base/#include) evaluates the contents of the input source file, `Include.jl`, in the notebook's global scope. The `Include.jl` file sets paths, loads required external packages, etc. For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/). 

Let's set up our code environment:

In [1]:
include(joinpath(@__DIR__, "Include-solution.jl")); # include the Include.jl file

In addition to standard Julia libraries, we'll also use [the `VLDataScienceMachineLearningPackage.jl` package](https://github.com/varnerlab/VLDataScienceMachineLearningPackage.jl). Check out [the documentation](https://varnerlab.github.io/VLDataScienceMachineLearningPackage.jl/dev/) for more information on the functions, types, and data used in this material.

### Implementation
Let's implement the `experiment(...)` function that is used by our reinforcement learning agent to evaluate drug cocktail designs. This function will compute the utility of a given drug cocktail based on the Cobb-Douglas utility function and apply the necessary constraints.

> __What is going on in the `experiment(...)` function?__
> 
> This function takes in a context object, which contains information about the drug types, their costs, and levels. It also takes in two integers, `s` and `a`, which represent the state and action, respectively. The function computes the utility of the drug cocktail based on the Cobb-Douglas utility function and checks if the cocktail meets the budget and safety constraints. If the constraints are met, it returns the computed utility; otherwise, it returns a penalty value.

We need to make a few changes from the lab template to implement the bounds and safety constraints.

> __What changes are needed?__ 
>
> In all cases, we will implement the constraints using squared violations to penalize constraint violations more heavily. You'll need to compute the budget, safety, and bounds violations as squared terms and include them in the final utility calculation with their respective penalties. Remember: we want to maximize utility, so penalties should be subtracted from the utility value.

Implement the revised `experiment(...)` function in the code cell below:

In [2]:
function experiment(context::MyExperimentalDrugCocktailContext, s::Int64, a::Int64)
    throw(ErrorException("Ooooops! The experiment function is not implemented yet"))
end

experiment (generic function with 1 method)

### Constants
In this section, let's define some constants that will be used in our drug cocktail design problem. These constants include the number of drug types, their costs, and the levels of drug concentrations. See the comment next to each constant for its units, permitted values, and description.

In [3]:
K = 3; # number of drug types
m = 3;  # number of drug levels per type {high, nominal, low}
𝒜 = range(1, stop=2^K, step=1) |> collect; # action space: 2^K possible drug combinations (binary selection vector)
𝒮 = range(1, stop=m^K, step=1) |> collect; # state space: m^K possible dosage level configurations
W = 80.0; # weight of the patient in kg
B = 1000.0; # TODO: you can change the budget in USD
S = nothing; # TODO: need to set a safety constraint for maximum allowable dosage units: mg/kg-day

___

## Task 1: Setup the context model
In this task, we will set up the context model for our drug cocktail design problem. This involves defining a mutable struct that holds the necessary information about the drugs, their costs, and levels. 

> __Context model explanation:__ The context model stores the problem structure including the number of drug types `K`, the number of dosage levels `m` per drug, the drug effectiveness coefficients `γ` (which the agent does not know a priori), the unit costs per drug, the three dosage concentration levels (high, nominal, low) for each drug in mg/kg, the patient weight `W` in kg, and the total budget constraint `B` in USD. This encapsulates all the problem parameters that define the drug cocktail design environment.

We need to make a few changes from the lab template to implement the bounds and safety constraints.

> __What changes are needed?__
> 
> We need to add the safety limit `S` and the minimum and maximum dosage levels for each drug type to the context model. This will allow us to enforce the safety and bounds constraints during the experiment evaluation. Note: we'll need to update the `MyExperimentalDrugCocktailContext` struct to include these new fields and the factory function that creates instances of this struct.
>
> Generate some dummy values for these new fields to test your implementation. For example, does your implementation respond correctly when the safety limit is exceeded or when drug dosages are outside their specified bounds?

We save the context information in the `contextmodel::MyExperimentalDrugCocktailContext` variable below:

In [4]:
contextmodel = let 

    # initialize - 
    costs = Dict{Int64, Float64}(); # cost per unit concentration (e.g., USD/mg) of drug i
    levels = Dict{Int64, NamedTuple}(); # levels of drug concentrations for drug i
    γ = Array{Float64}(undef, K); # effectiveness coefficients for each drug type

    # generate random cost data -
    for i ∈ 1:K
        costs[i] = rand(0.1:0.1:1.0); # random cost between 0.1 and 1.0 USD/mg
    end

    # generate random level data -
    for i ∈ 1:K
        high = rand(5.0:1.0:10.0);    # high concentration level in mg/kg
        nominal = rand(2.0:0.1:4.9);  # nominal concentration level in mg/kg
        low = rand(1.0:0.1:1.9);      # low concentration level in mg/kg
        levels[i] = (high=high, nominal=nominal, low=low);
    end

    # generate effectiveness coefficients -
    for i ∈ 1:K
        γ[i] = randn(); # random normal efficacy value (negative: inhibitory, positive: excitatory)
    end

    # TODO: implement the bounds array here -
    throw(ErrorException("You need to implement the bounds array for each drug type"));

    # build the model -
    model = nothing; # TODO: build the MyExperimentalDrugCocktailContext model here. Dont forget to pass in all necessary parameters!
    throw(ErrorException("You need to build the MyExperimentalDrugCocktailContext model here"));
    
    model; # return the model
end;

ErrorException: You need to implement the bounds array for each drug type

Let's examine the randomly generated problem parameters for our drug cocktail design instance. This table displays the unit costs, dosage concentration levels, and effectiveness coefficients for each drug type:

In [5]:
let
    # initialize -
    df = DataFrame();
    
    # extract data from context model -
    for i ∈ 1:K
        row_df = (
            drug = i,
            unit_cost = contextmodel.cost[i],
            low_conc = contextmodel.levels[i].low,
            nominal_conc = contextmodel.levels[i].nominal,
            high_conc = contextmodel.levels[i].high,
            γ = contextmodel.γ[i]
        );
        push!(df, row_df);
    end
    
    # display the table -
    pretty_table(
        df;
        backend = :text,
        table_format = TextTableFormat(borders = text_table_borders__compact)
    );
end

UndefVarError: UndefVarError: `contextmodel` not defined in `Main`
Suggestion: add an appropriate import or assignment. This global was declared but not assigned.

___

## Task 2: Construct the Learning Agent Model
In this task, we will construct the learning agent model that evaluates different drug cocktail designs. This agent will run experiments using the `experiment(...)` function we implemented earlier and will learn to optimize the drug cocktail design over time to maximize utility while adhering to the constraints.

In [6]:
mylearningagent = let

    # initialize -
    γ = 0.95; # discount factor (for future rewards)
    α = 0.1;  # learning rate
    Q = Array{Float64}(undef, length(𝒮), length(𝒜)); # Q-value table

    # fill the Q-table with zeros -
    fill!(Q, 0.0); # fast way to fill an array with a value

    # build the learning agent model -
    model = build(MyQLearningAgentModel, (
        γ = γ, # discount factor
        α = α, # learning rate
        Q = Q, # Q-value table
        states = 𝒮, # state space
        actions = 𝒜 # action space
    ));

    model; # return the model
end

MyQLearningAgentModel([1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  18, 19, 20, 21, 22, 23, 24, 25, 26, 27], [1, 2, 3, 4, 5, 6, 7, 8], 0.95, 0.1, [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0])

___

## Task 3: Let's let the agent learn!
In this task, we will allow the learning agent to interact with the environment and learn from its experiences. The agent will use the `experiment(...)` function to evaluate different drug cocktail designs and update its policy based on the observed rewards. 

> __What is going on in this code cell?__ We call the `solve(...)` method with our Q-learning agent, the context model, and the `experiment(...)` function as the world model. The agent will run up to 10,000 iterations, exploring different state-action pairs using epsilon-greedy selection. For each iteration, it takes an action (selecting drugs and dosages), observes the reward from the `experiment(...)` function (utility minus budget violation penalties), and updates the Q-value table using the temporal difference learning rule. The algorithm terminates when the maximum change in Q-values falls below the convergence tolerance δ = 0.0001 or when the maximum number of iterations is reached.

The `result` variable stores the trained Q-learning model containing the converged Q-value table `result.Q`, which encodes the expected cumulative reward for each state-action pair after learning.

In [7]:
result = solve(mylearningagent, contextmodel; 
    maxsteps = 10000, δ = 0.0001, worldmodel = experiment);

UndefVarError: UndefVarError: `contextmodel` not defined in `Main`
Suggestion: add an appropriate import or assignment. This global was declared but not assigned.

In [8]:
result.Q

UndefVarError: UndefVarError: `result` not defined in `Main`
Suggestion: add an appropriate import or assignment. This global was declared but not assigned.

Let's extract the policy from our trained agent.

In [9]:
π(s) = mypolicy(result.Q)[s]; # define a policy function that returns the best action for state s

What's in the policy? Let's pick the best action for each state according to the learned Q-values. We'll display these in a table.

> __What's going on in this table?__ For a given state `s` (representing a specific dosage level configuration), we extract the optimal action `a` from the learned policy. We then decode the action into a binary vector to identify which drugs are selected, and decode the state to determine the dosage level (high, nominal, or low) for each selected drug. 
> 
> The table displays the drug index, dosage level, unit cost, concentration levels available, total spending for that drug (concentration × patient weight × unit cost), the effectiveness coefficient γ, and the Q-value for this state-action pair. This shows the recommended drug cocktail composition that maximizes expected utility while respecting budget constraints for each state (dose combinations).

The table reveals which drugs the agent recommends including in the cocktail, at which dosage levels, and the expected value of following this recommendation from the given state.

In [10]:
let

    # initialize -
    df = DataFrame();
    s = 15; # what state do we want to look at?
    a = π(s); # this gives me the best action *index* for state s
    N = 2^K; # number of actions
    M = m^K; # number of states
    γ = contextmodel.γ; # effectiveness coefficients for each drug type

    aᵢ = digits(a, base=2, pad=K); # action vector representation
    if (a == N)
        aᵢ = digits(a-1, base=2, pad=K); # adjust for all-drug case
    end
    S₊ = findall(x -> x == 1, aᵢ); # indices of drugs being administered

    for i ∈ eachindex(S₊)
        drug_index = S₊[i];
        level_index = digits(s, base=m, pad=K)[drug_index];

        level_str = "";
        if (level_index == 0)
            level_str = "high";
        elseif (level_index == 1)
            level_str = "nominal";
        elseif (level_index == 2)
            level_str = "low";
        end
        n = contextmodel.levels[drug_index];
        spend = 0.0;
        if level_index == 0
            spend = (W*n.high)*contextmodel.cost[drug_index];
        elseif level_index == 1
            spend = (W*n.nominal)*contextmodel.cost[drug_index];
        elseif level_index == 2
            spend = (W*n.low)*contextmodel.cost[drug_index];
        end

        push!(df, (
            state = s,
            action = a,
            drug = drug_index,
            level = level_str,
            unitcost = contextmodel.cost[drug_index],
            concentration = n,
            spend = spend,
            γ = γ[drug_index],
            q_value = result.Q[s, a]
        ));
    end

    # make a pretty table -
    pretty_table(
        df;
        backend = :text,
        fit_table_in_display_horizontally = false,
        table_format = TextTableFormat(borders = text_table_borders__compact)
    );

end

UndefVarError: UndefVarError: `result` not defined in `Main`
Suggestion: add an appropriate import or assignment. This global was declared but not assigned.

___

## Discussion
This problem uses randomly generated parameters, so we're interested in checking whether our implementation responds reasonably to edge cases. Let's explore a few scenarios that test the Q-learning agent's behavior under different constraint conditions.

### Test case 1: Small budget constraint

> __Test:__ 
> What happens when the budget is so small that no drug combination is feasible? Set a small budget value, e.g., $B = 50$ or $B = 100$ USD, and retrain the agent with this new budget constraint. Examine the learned policy to see which drugs and dosages are selected. 

Check whether the policy selects cheaper drugs over expensive ones and whether the selected dosage levels are predominantly low rather than high. 

__Answer__: Describe what happened in test case 1 here ....

In [11]:
did_I_do_test_case_1 = false; # set to true if you did test case 1

### Test case 2: Negative efficacy coefficients

> __Test:__ 
> What happens when the most cost-effective drug has negative efficacy? Set the efficacy coefficients $\gamma$ such that at least one drug has a strongly negative value, e.g., $\gamma = [1.5, -2.0, 0.8]$. Retrain the agent with these efficacy values and examine which drugs appear in the learned policy.

Check whether the learned policy systematically excludes drugs with negative $\gamma$ values.

__Answer__: Describe what happened in test case 2 here ....

In [12]:
did_I_do_test_case_2 = false; # set to true if you did test case 2

### Test case 3: Tight safety constraint

> __Test:__ 
> What happens when even the minimum dosage of a single drug exceeds the safety limit? Set a low safety limit, e.g., $S = 10$ or $S = 5$ mg/kg-day, while keeping the budget generous. Retrain the agent with this tight safety constraint and analyze the resulting policy.

Check whether the policy respects the safety constraint by keeping $\sum_{i}n_{i} \leq S$ even when the budget allows for expensive high-dose drugs.

__Answer__: Describe what happened in test case 3 here .... 

In [13]:
did_I_do_test_case_3 = false; # set to true if you did test case 3

___

## Summary
In this lab, we applied Q-learning to solve the drug cocktail design problem, learning optimal policies for selecting drug combinations and dosage levels under budget constraints.

> __Key Takeaways:__
> 
> * **Q-learning for combinatorial optimization**: We implemented a Q-learning agent that iteratively estimates state-action value functions through temporal difference updates, converging to an optimal policy without requiring explicit knowledge of transition probabilities or reward functions. The algorithm balanced exploration (trying new drug combinations) with exploitation (selecting known effective combinations) using an epsilon-greedy strategy with a decaying exploration rate.
> * **MDP formulation for drug design**: We encoded the drug cocktail problem as a Markov decision process with states representing dosage level configurations (high, nominal, low for each drug type), actions representing binary drug selection vectors, and rewards computed from Cobb-Douglas utility functions penalized for budget violations. This formulation captured both therapeutic effectiveness (through preference parameters γᵢ) and resource constraints (through budget limits).
> * **Policy extraction and interpretation**: We derived the optimal policy by selecting argmax actions from the converged Q-value table, mapping each state to the drug combination that maximizes expected cumulative reward. The learned policy encoded both drug selection decisions (which drugs to include) and dosage optimization (which concentration levels to use), demonstrating how reinforcement learning discovers structured solutions to constrained combinatorial problems.

Q-learning provides a framework for sequential decision-making in optimization problems where the relationship between actions and outcomes must be learned through experience rather than analytical derivation.
___

## Tests
The code block below shows how we implemented the tests and what we are testing. In these tests, we check values in your notebook and give feedback on which items are correct, missing, etc.

In [14]:
@testset verbose = true "CHEME 5800 PS6 Test Suite" begin

    @testset "Experiment Function Tests" begin
        # Test 1: Check if experiment function exists and is callable
        @test isdefined(Main, :experiment) == true
        
        # Test 2: Check experiment function signature (should accept context, state, action)
        @test hasmethod(experiment, (MyExperimentalDrugCocktailContext, Int64, Int64)) == true
        
        # Test 3: Test experiment returns a tuple (s', reward)
        test_result = experiment(contextmodel, 1, 1)
        @test isa(test_result, Tuple) == true
        @test length(test_result) == 2
        
        # Test 4: Check that new state is within valid range
        mid_state = min(5, m^K)  # Use a state that exists regardless of dimension
        mid_action = min(3, 2^K)  # Use an action that exists regardless of dimension
        s_prime, reward = experiment(contextmodel, mid_state, mid_action)
        @test s_prime >= 1 && s_prime <= m^K
        
        # Test 5: Check that reward is a float
        @test isa(reward, Float64) == true
        
        # Test 6: Test budget constraint penalty application
        # High-cost scenario should produce different reward than low-cost
        r1 = experiment(contextmodel, 1, 2^K)[2]  # all drugs (expensive)
        r2 = experiment(contextmodel, 1, 1)[2]    # one drug (cheaper)
        @test isa(r1, Float64) == true && isa(r2, Float64) == true
        
        # Test 7: Test state transition logic
        # Next state should be valid regardless of transition strategy (wrap, random, etc.)
        s_last, _ = experiment(contextmodel, m^K, 1)
        @test s_last >= 1 && s_last <= m^K  # Should return a valid state
    end

    @testset "Context Model Tests" begin
        # Test 8: Check if contextmodel is defined
        @test isdefined(Main, :contextmodel) == true
        
        # Test 9: Check contextmodel type
        @test isa(contextmodel, MyExperimentalDrugCocktailContext) == true
        
        # Test 10: Check required fields in contextmodel
        @test hasfield(typeof(contextmodel), :K) == true
        @test hasfield(typeof(contextmodel), :m) == true
        @test hasfield(typeof(contextmodel), :γ) == true
        @test hasfield(typeof(contextmodel), :B) == true
        @test hasfield(typeof(contextmodel), :cost) == true
        @test hasfield(typeof(contextmodel), :levels) == true
        @test hasfield(typeof(contextmodel), :W) == true
        @test hasfield(typeof(contextmodel), :S) == true
        @test hasfield(typeof(contextmodel), :bounds) == true
        
        # Test 11: Check K is positive and reasonable
        @test contextmodel.K > 0
        @test contextmodel.K == K
        
        # Test 12: Check m is positive and reasonable
        @test contextmodel.m > 1
        @test contextmodel.m == m
        
        # Test 13: Check γ array has correct length
        @test length(contextmodel.γ) == contextmodel.K
        
        # Test 14: Check cost dictionary has K entries
        @test length(contextmodel.cost) == contextmodel.K
        
        # Test 15: Check levels dictionary has K entries
        @test length(contextmodel.levels) == contextmodel.K
        
        # Test 16: Check each drug has high, nominal, low levels
        for i in 1:contextmodel.K
            @test haskey(contextmodel.levels, i) == true
            @test haskey(contextmodel.levels[i], :high) == true
            @test haskey(contextmodel.levels[i], :nominal) == true
            @test haskey(contextmodel.levels[i], :low) == true
            # Verify ordering: low < nominal < high
            @test contextmodel.levels[i].low < contextmodel.levels[i].nominal < contextmodel.levels[i].high
            # Check all values are positive
            @test contextmodel.levels[i].low > 0.0
            @test contextmodel.levels[i].nominal > 0.0
            @test contextmodel.levels[i].high > 0.0
        end
        
        # Test 17: Check bounds array dimensions
        @test size(contextmodel.bounds) == (contextmodel.K, 2)
        
        # Test 18: Check budget value is defined and positive
        @test contextmodel.B > 0.0
        
        # Test 19: Check patient weight is defined and positive
        @test contextmodel.W > 0.0
        
        # Test 20: Check safety constraint is defined and positive
        @test contextmodel.S > 0
    end

    @testset "Learning Agent Tests" begin
        # Test 21: Check if mylearningagent is defined
        @test isdefined(Main, :mylearningagent) == true
        
        # Test 22: Check agent type
        @test isa(mylearningagent, MyQLearningAgentModel) == true
        
        # Test 23: Check required fields
        @test hasfield(typeof(mylearningagent), :γ) == true
        @test hasfield(typeof(mylearningagent), :α) == true
        @test hasfield(typeof(mylearningagent), :Q) == true
        @test hasfield(typeof(mylearningagent), :states) == true
        @test hasfield(typeof(mylearningagent), :actions) == true
        
        # Test 24: Check discount factor γ is valid (0 < γ < 1)
        @test mylearningagent.γ > 0.0 && mylearningagent.γ < 1.0
        
        # Test 25: Check learning rate α is valid (0 < α ≤ 1)
        @test mylearningagent.α > 0.0 && mylearningagent.α ≤ 1.0
        
        # Test 26: Check Q-table dimensions
        @test size(mylearningagent.Q) == (length(𝒮), length(𝒜))
        
        # Test 27: Check Q-table is properly initialized (should be numeric)
        @test isa(mylearningagent.Q, Array{Float64}) == true
        @test all(isfinite.(mylearningagent.Q)) == true
        
        # Test 28: Check states array
        @test length(mylearningagent.states) == length(𝒮)
        @test mylearningagent.states == 𝒮
        
        # Test 29: Check actions array
        @test length(mylearningagent.actions) == length(𝒜)
        @test mylearningagent.actions == 𝒜
    end

    @testset "Q-Learning Results Tests" begin
        # Test 30: Check if result is defined
        @test isdefined(Main, :result) == true
        
        # Test 31: Check result has Q field
        @test hasfield(typeof(result), :Q) == true
        
        # Test 32: Check Q-table dimensions match problem size
        @test size(result.Q) == (length(𝒮), length(𝒜))
        
        # Test 33: Q-table should have non-zero values after learning
        @test any(result.Q .!= 0.0) == true
        
        # Test 34: Check Q-values are finite (no NaN or Inf)
        @test all(isfinite.(result.Q)) == true
        
        # Test 35: Check that Q-values span a reasonable range
        max_q = maximum(result.Q)
        min_q = minimum(result.Q)
        @test max_q > min_q  # Should have learned different values
        
        # Test 36: Check that Q-table is numeric
        @test isa(result.Q, Array{Float64}) == true
    end

    @testset "Policy Extraction Tests" begin
        # Test 37: Check if policy function π is defined
        @test isdefined(Main, :π) == true
        
        # Test 38: Check if mypolicy function exists
        @test isdefined(Main, :mypolicy) == true
        
        # Test 39: Policy should return valid action for state 1
        action_1 = π(1)
        @test action_1 >= 1 && action_1 <= 2^K
        
        # Test 40: Policy should return integers
        test_state_idx = min(5, m^K)
        @test isa(π(test_state_idx), Integer) == true
        
        # Test 41: Test policy for multiple states
        num_test_states = min(6, m^K)  # Test up to 6 states if available
        test_states = collect(1:num_test_states)
        for test_state in test_states
            action = π(test_state)
            @test action >= 1 && action <= 2^K
        end
        
        # Test 42: Policy should be deterministic (same state → same action)
        test_state = min(7, m^K)
        action_first = π(test_state)
        action_second = π(test_state)
        @test action_first == action_second
        
        # Test 43: mypolicy should return an array of actions
        policy_array = mypolicy(result.Q)
        @test isa(policy_array, Array) == true
        @test length(policy_array) == m^K
        
        # Test 44: All policy actions should be valid
        @test all(policy_array .>= 1) == true
        @test all(policy_array .<= 2^K) == true
    end

    @testset "Constants and Setup Tests" begin
        # Test 45: Check K constant is defined and reasonable
        @test isdefined(Main, :K) == true
        @test K > 0 && K < 10  # Should be a reasonable number of drug types
        
        # Test 46: Check m constant is defined and reasonable
        @test isdefined(Main, :m) == true
        @test m > 1 && m < 10  # Should have multiple dosage levels
        
        # Test 47: Check action space 𝒜
        @test isdefined(Main, :𝒜) == true
        @test length(𝒜) == 2^K
        @test 𝒜[1] == 1 && 𝒜[end] == 2^K
        
        # Test 48: Check state space 𝒮
        @test isdefined(Main, :𝒮) == true
        @test length(𝒮) == m^K
        @test 𝒮[1] == 1 && 𝒮[end] == m^K
        
        # Test 49: Check W (patient weight) is defined and reasonable
        @test isdefined(Main, :W) == true
        @test W > 0.0 && W < 500.0  # Reasonable weight range in kg
        
        # Test 50: Check B (budget) is defined and positive
        @test isdefined(Main, :B) == true
        @test B > 0.0
        
        # Test 51: Check S (safety constraint) is defined and positive
        @test isdefined(Main, :S) == true
        @test S > 0
    end

    @testset "Integration Tests" begin
        # Test 52: Run a complete experiment cycle
        s_initial = 1
        a_test = min(3, 2^K)
        s_new, reward = experiment(contextmodel, s_initial, a_test)
        @test s_new >= 1 && s_new <= m^K  # Next state should always be valid
        @test isa(reward, Float64) == true
        
        # Test 53: Verify Q-learning improved over initial values
        # After learning, Q-values should differ from initial (all zeros)
        @test sum(abs.(result.Q)) > 0.0
        
        # Test 54: Check that learned policy prefers non-trivial actions
        # At least some states should have different actions (policy diversity)
        policy_diversity = length(unique(mypolicy(result.Q)))
        @test policy_diversity >= 1  # Should have at least one action
        
        # Test 55: Test consistency between Q-values and policy
        # The policy should select actions with maximum Q-values
        num_random_tests = min(5, m^K)  # Test up to 5 random states
        test_states = rand(1:m^K, num_random_tests)
        for test_state in test_states
            policy_action = π(test_state)
            q_values_for_state = result.Q[test_state, :]
            max_q_action = argmax(q_values_for_state)
            @test policy_action == max_q_action
        end
    end

    @testset "Completion Confirmation Tests" begin
        # Test 56: Confirm completion of test case 1
        @test did_I_do_test_case_1 == true
        
        # Test 57: Confirm completion of test case 2
        @test did_I_do_test_case_2 == true
        
        # Test 58: Confirm completion of test case 3
        @test did_I_do_test_case_3 == true
    end
end

Experiment Function Tests: Error During Test at /Users/jeffreyvarner/Desktop/julia_work/CHEME-5800-Fall-2025/PS6-CHEME-5800-TEMPLATE-Fall-2025/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y123sZmlsZQ==.jl:3
  Got exception outside of a @test
  UndefVarError: `contextmodel` not defined in `Main`
  Suggestion: add an appropriate import or assignment. This global was declared but not assigned.
  Stacktrace:
    [1] macro expansion
      @ ~/Desktop/julia_work/CHEME-5800-Fall-2025/PS6-CHEME-5800-TEMPLATE-Fall-2025/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y123sZmlsZQ==.jl:11 [inlined]
    [2] macro expansion
      @ ~/.julia/juliaup/julia-1.12.1+0.aarch64.apple.darwin14/share/julia/stdlib/v1.12/Test/src/Test.jl:1776 [inlined]
    [3] macro expansion
      @ ~/Desktop/julia_work/CHEME-5800-Fall-2025/PS6-CHEME-5800-TEMPLATE-Fall-2025/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y123sZmlsZQ==.jl:5 [inlined]
    [4] macro expansion
      @ ~/.julia/juliaup/julia-1.12.1+0.aarc

Test.TestSetException: Some tests did not pass: 35 passed, 5 failed, 27 errored, 0 broken.